In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install pyserini==0.22.0
!pip install faiss-cpu

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

In [ ]:
import json
import re
from bs4 import BeautifulSoup

with open('/content/drive/MyDrive/train_v7.json', 'r', encoding = 'utf-8-sig') as f:
  data_list1 = json.load(f)

with open('/content/drive/MyDrive/dev_v7.json', 'r', encoding = 'utf-8-sig') as f:
  data_list2 = json.load(f)


dict_data_list = data_list1 + data_list2

corpus_path = '/content/drive/MyDrive/fa'

new_data_list = []
exceptions_list = []
multiple_occurence = []

for each in os.listdir(corpus_path):
  if each != 'makarem.xml':
    file_path = corpus_path + '/' + each
    with open(file_path, 'r', encoding="utf8") as f:
      content = f.read()
    soup= BeautifulSoup(content, 'xml')
    for qa in dict_data_list:
      question = qa['question']
      answer = qa['answers'][0]['text']
      verse_id = qa['pq_id']

      try:
        translated_verse = soup.find(id=verse_id).contents[0]
        translated_verse = re.sub("[\(\[].*?[\)\]]", "", translated_verse)
        translated_verse = re.sub(" +", " ", translated_verse)
      except:
        exceptional_data = {}
        exceptional_data['verse_id'] = verse_id
        exceptional_data['translation_file'] = each
        exceptions_list.append(exceptional_data)

      if answer in translated_verse:
        new_data = {}
        new_data['context'] = translated_verse
        new_data['pq_id'] = qa['pq_id']
        new_data['question'] = question
        new_data['translation_file'] = each

        if translated_verse.count(answer) == 1:
          start_char = translated_verse.find(answer)
          answers_list = []
          answer_dict = {}
          answer_dict['text'] = answer
          answer_dict['start_char'] = start_char
          answers_list.append(answer_dict)
          new_data['answers'] = answers_list
          new_data_list.append(new_data)

        else:
          new_data['answers'] = answer
          new_data['freq'] = translated_verse.count(answer)
          multiple_occurence.append(new_data)

      else:
        continue

In [ ]:
translations_list = []

for each in os.listdir(corpus_path):
  translation = each
  translation = translation.removesuffix('.xml')
  if translation != 'makarem':
    translations_list.append(translation)

files_dict = {}
for each in translations_list:
  files_dict[each] = []

for qa in new_data_list:
  for each in translations_list:
    if each == qa['translation_file'].removesuffix('.xml'):
      files_dict[each].append(qa)
      continue


In [ ]:
extra_translation_path = '/content/drive/MyDrive/extra_translations'

for each in translations_list:
  os.mkdir(os.path.join(extra_translation_path,each))

for key, value in files_dict.items():
  split_percent = int(round(0.8*(len(value))))
  train = value[:split_percent]
  dev = value[split_percent:]

  train_path = extra_translation_path + '/' + key + '/' + key + '_train.json'
  dev_path = extra_translation_path + '/' + key + '/' + key + '_dev.json'

  with open(train_path, 'w') as tout:
    json.dump(train, tout)

  with open(dev_path, 'w') as dout:
    json.dump(dev, dout)


# creating extra translations DPR

a function to write documents in jsonl format

In [ ]:
def dump_jsonl(data, output_path, append=False):
    """
    Write list of objects to a JSON lines file.
    """
    mode = 'a+' if append else 'w'
    with open(output_path, mode, encoding='utf-8') as f:
        for line in data:
            json_record = json.dumps(line, ensure_ascii=False)
            f.write(json_record + '\n')
    print('Wrote {} records to {}'.format(len(data), output_path))

separately creating an index for each translation

In [ ]:
from bs4 import BeautifulSoup
import json
import os
import re

corpus_path = '/content/drive/MyDrive/fa'

for each in os.listdir(corpus_path):
  documents = []
  file_path = corpus_path + '/' + each
  with open(file_path, 'r', encoding="utf8") as f:
    content = f.read()
  soup= BeautifulSoup(content, 'xml')

  verses = soup.find_all("s")
  for verse in verses:
    doc_info = {}
    id = verse.get('id')
    doc_info['id'] = id
    content = re.sub("[\]]", "", verse.contents[0])
    content = re.sub(" +", " ", content)
    doc_info['contents'] = content
    documents.append(doc_info)

  input_path = '/content/drive/MyDrive/tests/resources'
  index_name = each.removesuffix('.xml') + '_collection_jsonl'
  os.mkdir(os.path.join(input_path,index_name))

  output_path = input_path + '/' + index_name
  dump_jsonl(documents, output_path + '/' + each.removesuffix('.xml') + '.jsonl')

  index_path = '/content/drive/MyDrive/indexes/' + index_name


  ! python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input $output_path \
  --language fa \
  --index $index_path \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

  print("**********************************************")

2024-01-26 13:17:08,364 INFO  [main] index.IndexCollection (IndexCollection.java:380) - Setting log level to INFO
2024-01-26 13:17:08,366 INFO  [main] index.IndexCollection (IndexCollection.java:383) - Starting indexer...
2024-01-26 13:17:08,367 INFO  [main] index.IndexCollection (IndexCollection.java:384) - ============ Loading Parameters ============
2024-01-26 13:17:08,367 INFO  [main] index.IndexCollection (IndexCollection.java:385) - DocumentCollection path: /content/drive/MyDrive/tests/resources/ansarian_collection_jsonl
2024-01-26 13:17:08,368 INFO  [main] index.IndexCollection (IndexCollection.java:386) - CollectionClass: JsonCollection
2024-01-26 13:17:08,369 INFO  [main] index.IndexCollection (IndexCollection.java:387) - Generator: DefaultLuceneDocumentGenerator
2024-01-26 13:17:08,369 INFO  [main] index.IndexCollection (IndexCollection.java:388) - Threads: 1
2024-01-26 13:17:08,369 INFO  [main] index.IndexCollection (IndexCollection.java:389) - Language: fa
2024-01-26 13:17:

converting QA datasets of each translation into DPR format

In [ ]:
from pyserini.search.lucene import LuceneSearcher
import random
import json

for each in os.listdir('/content/drive/MyDrive/extra_translations'):
  train_path = '/content/drive/MyDrive/extra_translations/' + each + '/' + each + '_train.json'
  dev_path = '/content/drive/MyDrive/extra_translations/' + each + '/' + each + '_dev.json'

  with open(train_path, 'r', encoding = 'utf-8-sig') as f:
    train_data = json.load(f)

  with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
    dev_data = json.load(f)

  dict_data_list  = train_data + dev_data

  index_path = '/content/drive/MyDrive/indexes/' + each + '_collection_jsonl'
  searcher = LuceneSearcher(index_path)
  searcher.set_language('fa')

  dpr_data_list = []

  for qa in dict_data_list:
    question = qa['question']
    dpr_data = {}
    dpr_data['dataset'] = 'Quran_QA'
    dpr_data['question'] = qa['question']
    dpr_data['answers'] = qa['answers']
    dpr_data['positive_ctxs'] = [{'title': '' , 'text': qa['context'],
                                  'score': 1000, 'title_score':1, 'passage_id':qa['pq_id']}]
    dpr_data['negative_ctxs'] = []

    hits = searcher.search(question)
    hard_negatives = []

    for i in range(len(hits)):
      verse_data = {}
      verse_id = hits[i].docid
      doc = searcher.doc(hits[i].docid)
      doc_dict = json.loads(doc.raw())
      verse_content = doc_dict['contents']
      if qa['answers'][0]['text'] not in verse_content and qa['pq_id'] != verse_id.split('_')[0]:
        verse_data['passage_id'] = verse_id
        verse_data['text'] = verse_content
        verse_data['title'] = ''
        verse_data['score'] = hits[i].score
        verse_data['title_score'] = 0
        hard_negatives.append(verse_data)

      if len(hard_negatives) == 5:
        break

    dpr_data['hard_negative_ctxs'] = hard_negatives

    if len(dpr_data['hard_negative_ctxs']) == 5:
      dpr_data_list.append(dpr_data)

    del dpr_data, hard_negatives, hits

  split_percent = int(round(0.8*(len(dpr_data_list))))

  # new_path = '/content/drive/MyDrive/extra_translation_DPR/' + each
  os.mkdir(os.path.join('/content/drive/MyDrive/extra_translations_DPR/',each))
  train_path = '/content/drive/MyDrive/extra_translations_DPR/' + each + '/' + each + '_DPR_train.json'
  dev_path = '/content/drive/MyDrive/extra_translations_DPR/' + each + '/' + each + '_DPR_dev.json'

  with open(train_path, 'w') as f:
    json.dump(dpr_data_list[:split_percent], f, ensure_ascii = False)

  with open(dev_path, 'w') as f:
    json.dump(dpr_data_list[split_percent:], f, ensure_ascii = False)

In [ ]:
for each in os.listdir('/content/drive/MyDrive/extra_translations_DPR'):

  dev_path = '/content/drive/MyDrive/extra_translations_DPR/' + each + '/' + each + '_DPR_train.json'

  with open(dev_path, 'r', encoding = 'utf-8-sig') as f:
    dict_data_list = json.load(f)

  new_dict_data_list = []

  for data in dict_data_list:
    new_data = {}
    new_data['title'] = ''
    # try:
    new_data['paragraphs'] = [{'context': data['positive_ctxs'][0]['text'], 'qas':[{'answers':[{'answer_start':data['answers'][0]['start_char'], 'text':data['answers'][0]['text']}],'id':data['positive_ctxs'][0]['passage_id'], 'question': data['question'], 'is_impossible': False}]}]
    # except:
      # print(data)
    new_dict_data_list.append(new_data)

  final_data = {}
  final_data['data'] = new_dict_data_list

  output_path = '/content/drive/MyDrive/extra_translations_DPR/' + each + '/' + each + '_dev_SQuAD_format.json'
  with open(output_path, 'w') as fp:
    json.dump(final_data, fp, ensure_ascii = False)